# Import Librarry

In [ ]:
import tensorflow as tf                                     
import keras_preprocessing                                  
from keras_preprocessing import image                      
from keras_preprocessing.image import ImageDataGenerator   
import time                                                
import matplotlib.pyplot as plt                            
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf  
from keras.models import Model
from sklearn.metrics import classification_report
from keras.layers import Input, Convolution2D, MaxPooling2D,AveragePooling2D,GlobalAveragePooling2D, Dense, Dropout, Flatten, BatchNormalization, Activation, MaxPool2D

# Data Preparation

In [ ]:
from google.colab import drive
#Membuka akses ke google drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os 
fold_Normal    = os.path.join('/content/gdrive/MyDrive/Data/Normal')         
fold_Benign     = os.path.join('/content/gdrive/MyDrive/Data/Benign')          
fold_Insistu = os.path.join('/content/gdrive/MyDrive/Data/InSitu')      
fold_Invasive     = os.path.join('/content/gdrive/MyDrive/Data/Invasive')          

print('Jumlah total gambar Albrecht Dürer:', len(os.listdir(fold_Normal)))                          
print('Jumlah total gambar Alfred Sisley:', len(os.listdir(fold_Benign)))                            
print('Jumlah total gambar Amedeo Modigliani:', len(os.listdir(fold_Insistu)))                    
print('Jumlah total gambar Andrei Rublev:', len(os.listdir(fold_Invasive)))                            


print("Menampilkan nama file gambar:")                                     
file_Normal = os.listdir(fold_Normal)                                                        
print("Normal:",file_Normal[:5])                                                             
file_Benign = os.listdir(fold_Benign)                                                          
print("Benign:",file_Benign[:5])                                                               

file_Insitu = os.listdir(fold_Insistu)                                                  
print("Insitu:",file_Insitu[:5])                                                       

file_Invasive = os.listdir(fold_Invasive)                                                        
print("Invasive:",file_Invasive[:5])                                                                 

Jumlah total gambar Albrecht Dürer: 30
Jumlah total gambar Alfred Sisley: 30
Jumlah total gambar Amedeo Modigliani: 30
Jumlah total gambar Andrei Rublev: 30
Menampilkan nama file gambar:
Normal: ['n001.jpg', 'n002.jpg', 'n003.jpg', 'n004.jpg', 'n005.jpg']
Benign: ['b001.jpg', 'b002.jpg', 'b003.jpg', 'b004.jpg', 'b005.jpg']
Insitu: ['is001.jpg', 'is002.jpg', 'is003.jpg', 'is004.jpg', 'is005.jpg']
Invasive: ['iv001.jpg', 'iv002.jpg', 'iv003.jpg', 'iv004.jpg', 'iv005.jpg']


In [ ]:
BASIS_DIR = "/content/gdrive/MyDrive/Data"                        
training_datagen = ImageDataGenerator(                                            
      rescale = 1./255,                                                           
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>AUGMENTASI<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
	    rotation_range=40,                                                                    
      width_shift_range=0.2,                                                      
      height_shift_range=0.2,                                                     
      shear_range=0.2,                                                            
      zoom_range=0.2,                                                             
      horizontal_flip=True,                                                        
      fill_mode='nearest',                                                         
#>>>>>>>>>>>MEMECAH DATA MENJADI DATA TRAINING DAN DATA VALIDASI<<<<<<<<<<<<<<<<      
      validation_split=0.2                                                       
      )

In [ ]:
train_generator = training_datagen.flow_from_directory(                           
	BASIS_DIR,                                                                      
	target_size=(256,256),                                                          
	class_mode='categorical',                                                       
  shuffle=True,                                                                   
  subset='training'                                                               
  )

validation_generator = training_datagen.flow_from_directory(                      
	BASIS_DIR,                                                                      
	target_size=(256,256),                                                          
	class_mode='categorical',                                                       
  shuffle=True,
  subset='validation'                                                             
  )

Found 96 images belonging to 4 classes.
Found 24 images belonging to 4 classes.


In [ ]:
class WaktuTraining(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

callbacks = WaktuTraining()

# Create Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K

In [ ]:
# Creating Densenet201
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,48,32]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    model = Model(input, output)
    return model
input_shape = 256, 256, 3
n_classes = 4
model = densenet(input_shape,n_classes)


In [ ]:
inp1 = Input(shape=(256,256,3))
ResNetXt50=Convolution2D(1,kernel_size=(7,7),strides=2,padding='same',activation='relu')(inp1)
ResNetXt50=MaxPooling2D(pool_size=(3, 3),strides=2)(ResNetXt50)
#ResNetXt50=Dense(3)(ResNetXt50)
ResNetXt50=Convolution2D(128,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(128,kernel_size=(3,3),groups=32,padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(256,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
#ResNetXt50=Dense(4)(ResNetXt50)
ResNetXt50=Convolution2D(256,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(256,kernel_size=(3,3),groups=32,padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(512,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
#ResNetXt50=Dense(6)(ResNetXt50)
ResNetXt50=Convolution2D(512,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(512,kernel_size=(3,3),groups=32,padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(1024,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
#ResNetXt50=Dense(3)(ResNetXt50)
ResNetXt50=Convolution2D(1024,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(1024,kernel_size=(3,3),groups=32,padding='same',activation='relu')(ResNetXt50)
ResNetXt50=Convolution2D(2048,kernel_size=(1,1),padding='same',activation='relu')(ResNetXt50)
ResNetXt50=GlobalAveragePooling2D()(ResNetXt50)
flatten_Res=Flatten()(ResNetXt50)

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, decay=0.0005)

In [ ]:
model.compile(optimizer=opt, loss='CategoricalCrossentropy', metrics=['Accuracy'])

In [ ]:
class WaktuTraining(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

callbacks = WaktuTraining()

In [ ]:
history=model.fit(
    train_generator,                                                              #Data training hasil generasi
    steps_per_epoch=2,                                                            #Jumlah batch yang akan dieksekusi pada setiap epoch
    epochs=40,                                                                  #Jumlah perulangan komputasi(epoch)
    validation_data=validation_generator,                                         #Menampilkan akurasi pengujian data validasi
    validation_steps=5,                                                           #Jumlah batch yang akan dieksekusi pada setiap epoch
    verbose=2,
    callbacks=[callbacks]
    )

Epoch 1/40
2/2 - 88s - loss: 1.3823 - Accuracy: 0.3438 - val_loss: 1.3874 - val_Accuracy: 0.2500 - 88s/epoch - 44s/step
Epoch 2/40
2/2 - 2s - loss: 1.3413 - Accuracy: 0.2969 - 2s/epoch - 982ms/step
Epoch 3/40
2/2 - 2s - loss: 1.1971 - Accuracy: 0.5156 - 2s/epoch - 976ms/step
Epoch 4/40
2/2 - 2s - loss: 1.1512 - Accuracy: 0.4375 - 2s/epoch - 974ms/step
Epoch 5/40
2/2 - 2s - loss: 1.0565 - Accuracy: 0.5156 - 2s/epoch - 1s/step
Epoch 6/40
2/2 - 2s - loss: 0.9834 - Accuracy: 0.6406 - 2s/epoch - 1s/step
Epoch 7/40
2/2 - 2s - loss: 0.9603 - Accuracy: 0.6875 - 2s/epoch - 971ms/step
Epoch 8/40
2/2 - 2s - loss: 0.8909 - Accuracy: 0.7344 - 2s/epoch - 976ms/step
Epoch 9/40
2/2 - 2s - loss: 0.8753 - Accuracy: 0.7188 - 2s/epoch - 978ms/step
Epoch 10/40
2/2 - 2s - loss: 0.7798 - Accuracy: 0.7031 - 2s/epoch - 979ms/step
Epoch 11/40
2/2 - 2s - loss: 0.9342 - Accuracy: 0.6094 - 2s/epoch - 997ms/step
Epoch 12/40
2/2 - 2s - loss: 0.8166 - Accuracy: 0.6719 - 2s/epoch - 979ms/step
Epoch 13/40
2/2 - 2s - lo

In [ ]:
history=model.fit(
    train_generator,                                                              #Data training hasil generasi
    steps_per_epoch=2,                                                            #Jumlah batch yang akan dieksekusi pada setiap epoch
    epochs=50,                                                                  #Jumlah perulangan komputasi(epoch)
    validation_data=validation_generator,                                         #Menampilkan akurasi pengujian data validasi
    validation_steps=5,                                                           #Jumlah batch yang akan dieksekusi pada setiap epoch
    verbose=2,
    callbacks=[callbacks]
    )

Epoch 1/50
2/2 - 3s - loss: 0.3000 - Accuracy: 0.9375 - val_loss: 2.0371 - val_Accuracy: 0.2500 - 3s/epoch - 2s/step
Epoch 2/50
2/2 - 2s - loss: 0.3082 - Accuracy: 0.9688 - 2s/epoch - 990ms/step
Epoch 3/50
2/2 - 2s - loss: 0.3824 - Accuracy: 0.8281 - 2s/epoch - 988ms/step
Epoch 4/50
2/2 - 2s - loss: 0.3048 - Accuracy: 0.8906 - 2s/epoch - 997ms/step
Epoch 5/50
2/2 - 2s - loss: 0.3729 - Accuracy: 0.8906 - 2s/epoch - 990ms/step
Epoch 6/50
2/2 - 2s - loss: 0.4014 - Accuracy: 0.9062 - 2s/epoch - 999ms/step
Epoch 7/50
2/2 - 2s - loss: 0.3174 - Accuracy: 0.8906 - 2s/epoch - 1s/step
Epoch 8/50
2/2 - 2s - loss: 0.4468 - Accuracy: 0.8281 - 2s/epoch - 1s/step
Epoch 9/50
2/2 - 2s - loss: 0.3187 - Accuracy: 0.8750 - 2s/epoch - 997ms/step
Epoch 10/50
2/2 - 2s - loss: 0.3210 - Accuracy: 0.9219 - 2s/epoch - 993ms/step
Epoch 11/50
2/2 - 2s - loss: 0.2787 - Accuracy: 0.9375 - 2s/epoch - 996ms/step
Epoch 12/50
2/2 - 2s - loss: 0.3609 - Accuracy: 0.9219 - 2s/epoch - 994ms/step
Epoch 13/50
2/2 - 2s - loss:

In [ ]:
history=model.fit(
    train_generator,                                                              #Data training hasil generasi
    steps_per_epoch=2,                                                            #Jumlah batch yang akan dieksekusi pada setiap epoch
    epochs=30,                                                                  #Jumlah perulangan komputasi(epoch)
    validation_data=validation_generator,                                         #Menampilkan akurasi pengujian data validasi
    validation_steps=5,                                                           #Jumlah batch yang akan dieksekusi pada setiap epoch
    verbose=2,
    callbacks=[callbacks]
    )

Epoch 1/30
2/2 - 64s - loss: 1.5310 - Accuracy: 0.2188 - val_loss: 1.3862 - val_Accuracy: 0.2500 - 64s/epoch - 32s/step
Epoch 2/30
2/2 - 2s - loss: 1.2795 - Accuracy: 0.3281 - 2s/epoch - 970ms/step
Epoch 3/30
2/2 - 2s - loss: 1.2549 - Accuracy: 0.4531 - 2s/epoch - 958ms/step
Epoch 4/30
2/2 - 2s - loss: 1.1408 - Accuracy: 0.5156 - 2s/epoch - 970ms/step
Epoch 5/30
2/2 - 2s - loss: 1.0732 - Accuracy: 0.5938 - 2s/epoch - 972ms/step
Epoch 6/30
2/2 - 2s - loss: 1.0101 - Accuracy: 0.6719 - 2s/epoch - 974ms/step
Epoch 7/30
2/2 - 2s - loss: 0.9279 - Accuracy: 0.6250 - 2s/epoch - 975ms/step
Epoch 8/30
2/2 - 2s - loss: 0.8421 - Accuracy: 0.7031 - 2s/epoch - 978ms/step
Epoch 9/30
2/2 - 2s - loss: 0.8112 - Accuracy: 0.7188 - 2s/epoch - 975ms/step
Epoch 10/30
2/2 - 2s - loss: 0.9104 - Accuracy: 0.6250 - 2s/epoch - 976ms/step
Epoch 11/30
2/2 - 2s - loss: 0.8348 - Accuracy: 0.7031 - 2s/epoch - 970ms/step
Epoch 12/30
2/2 - 2s - loss: 0.7067 - Accuracy: 0.7812 - 2s/epoch - 1s/step
Epoch 13/30
2/2 - 2s -

In [ ]:
print(model.layers[1].weights)
print(model.layers[1].bias.numpy().shape)
print(model.layers[1].bias_initializer)

[<tf.Variable 'conv2d/kernel:0' shape=(7, 7, 3, 64) dtype=float32, numpy=
array([[[[-5.69248805e-05,  1.67542440e-03,  3.90384719e-02, ...,
          -6.94654300e-04,  2.51891539e-02, -4.10824753e-02],
         [ 2.12180987e-02, -3.76890525e-02,  1.52615644e-02, ...,
          -9.89584066e-03, -3.39079872e-02, -2.80059804e-03],
         [ 1.19195543e-02, -4.08870392e-02,  3.37783992e-02, ...,
           5.95553219e-03,  4.38668355e-02, -9.63324774e-03]],

        [[-1.54636512e-02,  1.55656952e-02,  2.46874094e-02, ...,
          -3.48801091e-02,  4.09116270e-03, -4.20963950e-02],
         [ 7.02489913e-03,  3.77090722e-02, -9.06948373e-03, ...,
           2.93879025e-03, -8.41322355e-03,  3.78029346e-02],
         [ 2.84613650e-02,  1.71968639e-02,  4.03867438e-02, ...,
          -2.15424015e-03, -1.07712336e-02,  2.72107888e-02]],

        [[ 3.61604244e-02,  1.74639057e-02, -2.61599515e-02, ...,
           7.24533945e-03, -2.21899264e-02, -2.20639613e-02],
         [-1.32221254e-02,

In [ ]:
Waktu_Training = callbacks.times
print('Waktu Training=', sum(Waktu_Training), "detik")

In [ ]:
model.save('model_skripsi.h5')

In [ ]:
Waktu_Training = callbacks.times
print('Waktu Training=', sum(Waktu_Training), "detik")

Waktu Training= 94.85205006599426 detik
